In [ ]:
import pandas as pd
import numpy as np
import cv2

import os

from matplotlib import pyplot as plt

from tqdm.auto import tqdm

save_dir = f'/kaggle/tmp/mask'
size = 512

os.makedirs(save_dir, exist_ok=True)

In [ ]:
train = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')

meta = pd.read_csv('../input/siim-covid19-resized-to-512px-png/meta.csv')
meta = meta[meta.split=='train'].reset_index(drop=True)

train['id'] = train.apply(lambda row: row.id.split('_')[0], axis=1)
train = train.rename(columns={'id': 'image_id'})
train_meta = pd.merge(train, meta, how='inner', on='image_id')
train_meta = train_meta.drop(['boxes', 'split', 'StudyInstanceUID'], axis=1)
train_meta.head()

# Show one mask image

In [ ]:
image_id = '000a312787f2'
i = int(train_meta[train_meta.image_id==image_id].index.values)

label = train_meta.loc[i, 'label'].split()
if label != 'none':
    label = label
    dim0 = train_meta.loc[i, 'dim0']
    dim1 = train_meta.loc[i, 'dim1']

    mask = np.zeros((size, size), dtype=np.uint8)
    for j in range(len(label) // 6):
        xmin = int(np.round(float(label[6*j+2])*size/dim1, 4))
        ymin = int(np.round(float(label[6*j+3])*size/dim0, 4))
        xmax = int(np.round(float(label[6*j+4])*size/dim1, 4))
        ymax = int(np.round(float(label[6*j+5])*size/dim0, 4))
        cv2.rectangle(mask, (xmin, ymin), (xmax, ymax), color=255, thickness=-1)

    plt.imshow(mask)

# Save all mask images

In [ ]:
for i in tqdm(range(len(train_meta))):
    label = train_meta.loc[i, 'label'].split()
    
    if label[0] == 'none':
        continue
    
    image_id = train_meta.loc[i, 'image_id']
    dim0 = train_meta.loc[i, 'dim0']
    dim1 = train_meta.loc[i, 'dim1']
    
    mask = np.zeros((size, size), dtype=np.uint8)
    for j in range(len(label) // 6):
        xmin = int(np.round(float(label[j*6+2])*size/dim1, 4))
        ymin = int(np.round(float(label[j*6+3])*size/dim0, 4))
        xmax = int(np.round(float(label[j*6+4])*size/dim1, 4))
        ymax = int(np.round(float(label[j*6+5])*size/dim0, 4))
        cv2.rectangle(mask, (xmin, ymin), (xmax, ymax), color=255, thickness=-1)
    cv2.imwrite(f'{save_dir}/{image_id}.png', mask)

In [ ]:
%%time
!tar -zcf mask.tar.gz -C "/kaggle/tmp/mask/" .